# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df['Max Temp'] = (city_data_df['Max Temp'] - 273.15) * 9/5 + 32


# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,monrovia,6.3005,-10.7969,81.860,88,36,1.14,LR,1710621655
1,1,cape girardeau,37.3834,-89.7168,66.794,35,69,5.77,US,1710621656
2,2,bilibino,68.0546,166.4372,-6.538,92,21,1.32,RU,1710621656
3,3,la passe,45.5549,-0.8967,58.010,82,100,3.61,FR,1710621656
4,4,grytviken,-54.2811,-36.5092,40.532,86,34,4.37,GS,1710621657


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",
    scale=0.5,
    color="City",
)

# Display the map
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df= city_data_df.loc[
    (city_data_df["Max Temp"] < 80) & 
    (city_data_df["Humidity"] < 30)&
    (city_data_df["Cloudiness"] < 50)&
    (city_data_df["Wind Speed"] < 10)
]

# Drop any rows with null values
ideal_weather_df=ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
43,43,anjad,22.0333,75.0500,79.394,23,0,1.66,IN,1710621672
47,47,illizi,26.4833,8.4667,67.802,22,0,3.19,DZ,1710621674
174,174,recani,31.2846,-4.2688,71.780,17,0,3.38,MA,1710621734
254,254,saugor,23.8333,78.7167,69.152,24,10,3.24,IN,1710621771
307,307,'alaqahdari dishu,30.4374,63.3130,67.496,25,49,2.44,AF,1710621794
308,308,hami,42.8000,93.4500,34.484,29,2,3.06,CN,1710621681
347,347,ghat,24.9647,10.1728,65.012,21,0,3.89,LY,1710621811
383,383,radhanpur,23.8333,71.6000,77.576,21,2,4.41,IN,1710621827
438,438,alaghsas,17.0187,8.0168,78.674,14,0,7.56,NE,1710621851
474,474,cauquenes,-35.9671,-72.3225,79.232,24,4,4.99,CL,1710621866


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, 
#country, coordinates, and humidity
hotel_df= ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
43,anjad,IN,22.0333,75.0500,23,
47,illizi,DZ,26.4833,8.4667,22,
174,recani,MA,31.2846,-4.2688,17,
254,saugor,IN,23.8333,78.7167,24,
307,'alaqahdari dishu,AF,30.4374,63.3130,25,
308,hami,CN,42.8000,93.4500,29,
347,ghat,LY,24.9647,10.1728,21,
383,radhanpur,IN,23.8333,71.6000,21,
438,alaghsas,NE,17.0187,8.0168,14,
474,cauquenes,CL,-35.9671,-72.3225,24,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000 
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    #get latitude, longitude from the DataFrame
    lat=row["Lat"]
    lng=row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address= requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address= name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
anjad - nearest hotel: No hotel found
illizi - nearest hotel: بوناقة
recani - nearest hotel: Hotel Merzouga
saugor - nearest hotel: No hotel found
'alaqahdari dishu - nearest hotel: No hotel found
hami - nearest hotel: Хами
ghat - nearest hotel: فندق تاسيلي
radhanpur - nearest hotel: No hotel found
alaghsas - nearest hotel: Hôtel Zagado
cauquenes - nearest hotel: Hotel Maule
aoulef - nearest hotel: دار الضياف
bilara - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
43,anjad,IN,22.0333,75.0500,23,No hotel found
47,illizi,DZ,26.4833,8.4667,22,بوناقة
174,recani,MA,31.2846,-4.2688,17,Hotel Merzouga
254,saugor,IN,23.8333,78.7167,24,No hotel found
307,'alaqahdari dishu,AF,30.4374,63.3130,25,No hotel found
308,hami,CN,42.8000,93.4500,29,Хами
347,ghat,LY,24.9647,10.1728,21,فندق تاسيلي
383,radhanpur,IN,23.8333,71.6000,21,No hotel found
438,alaghsas,NE,17.0187,8.0168,14,Hôtel Zagado
474,cauquenes,CL,-35.9671,-72.3225,24,Hotel Maule


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_hotel = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",
    scale=0.5,
    color="City",
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_hotel

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)